# Chapter 2 - Patters for Cleaner Python

## 2.1 Assertions

In [66]:
def apply_discount(product, discount):
    price = product["price"] * (1 - discount)
    assert 0 <= price <= product["price"]
    return price

In [67]:
apply_discount({"price": 200}, 0.1)

180.0

In [68]:
apply_discount({"price": 200}, 1.2)

AssertionError: 

Why not use exceptions instead of assertions? Because exceptions are meant to be handled, while assertions are meant to be used as a debugging aid. An assertion should never be raised unless there is a bug in the code. If an assertion fails, it indicates that there is a bug in the code. Exceptions, on the other hand, are meant to be handled and can be raised for a variety of reasons, including user input errors, network errors, etc.


### 2.1.1 Syntax
```python
assert_stmt ::= assert expression1 [',' expression2]
```

Here, `expression1` is the condition to be checked, and `expression2` is an optional message that will be printed if the assertion fails. If `expression1` evaluates to `False`, an `AssertionError` is raised with the message from `expression2`.
### 2.1.2 Example


```python
assert True, "This should never happen"

In [ ]:
# above is equivalent to below python code

expression1 = False
expression2 = "Message to come with assertion"
if __debug__:
    if not expression1:
        raise AssertionError(expression2)

AssertionError: Message to come with assertion

In [ ]:
# another use case
cond = "z"
if cond == "x":
    do_x()
elif cond == "y":
    do_y()
else:
    assert False, (
        "This shouldn't come but if this has come,something is not as expected"
        "contact the developer",
    )

AssertionError: ("This shouldn't come but if this has come,something is not as expectedcontact the developer",)

There are two caveats to know while using assertions:

1. **Do Not use to validate data**: Assertions can be disabled globally with the `-O` (optimize) flag or the `PYTHONOPTIMIZE` environment variable. This means that assertions should not be used for data validation or any other critical checks that need to be performed in production code.

```python
def delete_product(prod_id,user):
    assert user.is_admin, "Only admins can delete products"
    assert prod_id in products, "Product not found"
    del products[prod_id]
```

In above code, if the user is not an admin and runs the code with `-O` flag, the assertion will be skipped and the product will be deleted which is very dangerous. 

2. **Asserts that never fail**: 

Look at following code:

In [ ]:
assert (1 == 2, "This should fail")

<>:1: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<>:1: SyntaxWarning: assertion is always true, perhaps remove parentheses?
/var/folders/kc/cmchflms1qxgpgwvllhfc3080000gn/T/ipykernel_48494/953681201.py:1: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert (1 == 2, "This should fail")


Reason being when you pass tuple as the first argument to assert, it will always be true. This is because the tuple is evaluated as a truthy value, and the assertion will never fail. Newer versions of Python will raise a `SyntaxWarning` for this, but it is still a good practice to avoid using tuples in assertions.


## 2.2 Complacent Comma Placement

Smart formatter tools like `black` can help you avoid this problem by automatically formatting your code to ensure that commas are placed correctly. However, it is still important to be aware of how to commas placement can affect the readability of your code.

When you have a long list of items in lists or tuples it is a good practice to write each item on a new line and place the comma at the end of the line including the last item. This makes it easier to read and understand the code, and also makes it easier to add or remove items from the list in the future.

```python
# Bad
my_list = [
    "item1", "item2", "item3", "item4", "item5", "item6", "item7",
    "item8", "item9", "item10"
]

# Good
my_list = [
    "item1",
    "item2",
    "item3",
    "item4",
    "item5",
    "item6",
    "item7",
    "item8",
    "item9",
    "item10",
]
```


In [ ]:
names = [
    "Alice",
    "Bob",
    "Dilbert",
    "Jane",
    "Doe",
    "Sunil",
    "Bob",
    "Dilbert",
    "Jane",
    "Doe" "Denver",
]
names

['Alice',
 'Bob',
 'Dilbert',
 'Jane',
 'Doe',
 'Sunil',
 'Bob',
 'Dilbert',
 'Jane',
 'DoeDenver']

If you observe the above code, last two items in the list are concatenated. This merging of strings is a feature in python refereed to as "string literal concatenation". 

> String literal concatenation: Multiple adjacent string or bytes literals possibly using different types of quotes are allowed and their meaning is the same as their concatenation.

In [ ]:
my_str = (
    "This is a super long string that spans "
    "acorss different lines using different quotations "
    "although python does concantenate for us"
)
my_str

'This is a super long string that spans acorss different lines using different quotations although python does concantenate for us'

## 2.3 Context Managers and the `with` statement

> Benefit of using with? It helps simplify common resource management patterns by abstracting their functionality and allowing them to be factored out and reused.

In [89]:
with open("hello.txt", "w") as f:
    f.write("Hello")

Above is same as below code

In [91]:
f = open("hello.txt", "a")
try:
    f.write("hello")
finally:
    f.close()

In [ ]:
# this is bad

f = open("hello.txt", "w")
f.write("hello")
f.close()

Above is not the correct way as if any exception occurs in the code, release of the resource will not happen and the resource will be leaked. 


In [ ]:
#another threading example
import threading
some_lock = threading.Lock()

#Harmful
some_lock.acquire()

try:
    #Do something here
finally:
    some_lock.release()

#Better way

with some_lock:
    #Do Something here

In order to support with in your own classes, you need to implement two special methods: `__enter__` and `__exit__`. The `__enter__` method is called when the `with` statement is executed, and the `__exit__` method is called when the block of code inside the `with` statement is exited. 



In [93]:
class ManagedFile:
    def __init__(self, name):
        self.name = name

    def __enter__(self):
        self.file = open(self.name, "w")
        return self.file

    def __exit__(self, exc_type, exc_val, exc_tb):
        if self.file:
            self.file.close()


with ManagedFile("hello.txt") as f:
    f.write("Bye")

Above examples are class-based context managers. You can also create context managers using generator functions with the `contextlib` module.



In [ ]:
from contextlib import contextmanager


@contextmanager
def managed_file(filename):
    try:
        file = open(filename, "w")
        yield file
    finally:
        file.close()


with managed_file("hello.txt") as f:
    f.write("Hello again!@")
    f.write("Bye now ")

`managed_file()` is a generator that first acquires the resource. After that, it temporarily suspends its own execution and yields the resource so itcan be used by the caller. When the caller leaves the with context, the generator continues to execute so that any remaining clean-up steps can occur and the resource can get released back to the system.

In [99]:
class Indenter:
    def __init__(self):
        self.level = 0

    def __enter__(self):
        self.level += 1
        return self

    def print(self, message):
        print("    " * self.level + message)

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.level -= 1


with Indenter() as indent:
    indent.print("hi!")
    with indent:
        indent.print("hello")
        with indent:
            indent.print("Namaste")
    indent.print("bagunnara")

    hi!
        hello
            Namaste
    bagunnara


## 2.4 Underscores, Dunders and More

In broad, there are five underscore conventions in Python:

1. **Single leading underscore**: `_var` 
2. **Single trailing underscore**: `var_`
3. **Double leading underscore**: `__var`
4. **Double leading and trailing underscore**: `__var__`
5. **Single underscore**: `_`

### 2.4.1 Single leading underscore
The single leading underscore is only a convention and does not affect the behavior of the variable. It is used to indication to possibly another developer that the variable is intended for internal use only and should not be accessed directly. It is a weak "internal use" indicator. This convention is defined in PEP 8, the common style guide for Python code.



In [ ]:
class Test:
    def __init__(self):
        self.foo = 11
        self._bar = 13


t = Test()
print(t.foo)
print(t._bar)

11
13


Single leading underscore doesn't stop from accessing the varible as shown here. This is merely an agreed upon convention. However, leading underscore does impact how the variable is imported when using wildcard import syntax `from module import *`. Anyhow, it is a bad practice and to be avoided using wildcard imports as they make it unclear which names are present in the namespace

In [ ]:
from my_module import *

print(external_func())
print(_internal_func())

23


NameError: name '_internal_func' is not defined

In [ ]:
from my_module import external_func, _internal_func

print(external_func())
print(_internal_func())

23
24


### 2.4.2 Single trailing underscore

The single trailing underscore is used to avoid naming conflicts with Python keywords or built-in names. For example, if you want to use the name `class` as a variable name, you can use `class_` instead. This is also a convention and does not affect the behavior of the variable.

In [ ]:
def make_object(name,class):
    pass

SyntaxError: invalid syntax (1426137068.py, line 1)

In [ ]:
def make_object(name, class_):
    pass

### 2.4.3 Double leading underscore

A double leading underscore prefix causes the Python interpreter to rewrite the attribute name in order to avoid naming conflicts in subclasses. This is known as name mangling. The interpreter changes the name of the variable in a way that makes it harder to create subclasses that accidentally override the private attributes and methods.

In [ ]:
class Test:
    def __init__(self):
        self.foo = 42
        self._bar = 24
        self.__baz = 242


t = Test()
dir(t)

['_Test__baz',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_bar',
 'foo']

You see `foo`, `bar` and `_Test__baz` in place of `__baz` in the output. This is because the interpreter rewrote the name of the variable to include the class name as a prefix. Python does this to protect the variable from being overridden in subclasses. 

In [ ]:
class ExtendedTest(Test):
    def __init__(self):
        super().__init__()
        self.foo = "overridden"
        self._bar = "overridden"
        self.__baz = "overridden"


t2 = ExtendedTest()
print(t2.foo)
print(t2._bar)
print(t2.__baz)

overridden
overridden


AttributeError: 'ExtendedTest' object has no attribute '__baz'

In [78]:
dir(t2)

['_ExtendedTest__baz',
 '_Test__baz',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_bar',
 'foo']

In [79]:
print(t2._ExtendedTest__baz)
print(t2._Test__baz)

overridden
242


Double underscore name mangling is fully transparent to the programmer. Look at the below example:

In [83]:
class ManglingTest:
    def __init__(self):
        self.__mangled = "hello"

    def get_mangled(self):
        return self.__mangled


mt = ManglingTest()
print(mt.get_mangled())
print(mt._ManglingTest__mangled)
print(mt.__mangled)

hello
hello


AttributeError: 'ManglingTest' object has no attribute '__mangled'

In [84]:
# does it apply to methods?


class MangledMethod:
    def __method(self):
        return 42

    def call_mangled(self):
        return self.__method()


mm = MangledMethod()

print(mm.__method())

AttributeError: 'MangledMethod' object has no attribute '__method'

In [85]:
mm._MangledMethod__method()

42

In [86]:
mm.call_mangled()

42

In [88]:
# another example to better understand name mangling

_MangledGlobal__mangled = 23


class MangledGlobal:
    def test(self):
        return __mangled


MangledGlobal().test()

23

Above example demonstrates that name mangling isn't tied to class attributes sepcifically. It applies to any name starting with two underscore characters that is used in the context of a class. 

Double underscores is often referred to as "dunder" (short for "double underscore"). For example, `__init__` is often referred to as "dunder init" and `__baz` is often referred to as "dunder baz".


### 2.4.4 Double leading and trailing underscore

Variables with double leading and trailing underscores are reserved for special use in the language. These are also referred to as "dunder" variables. These are left unscathed by the python interpreter and are not mangled. These variables are used to define special methods in classes, such as `__init__`, `__str__`, `__repr__`, etc. These methods are called "magic methods" and are used to define the behavior of the class in certain situations. For example, the `__init__` method is called when an object is created, and the `__str__` method is called when the object is converted to a string.

In [ ]:
class PrefixPostfixTest:
    def __init__(self):
        self.__bam__ = 42


PrefixPostfixTest().__bam__

42

It is best to avoid using them in your own programs to avoid collisions with future changes to the python language

### 2.4.5 Single underscore

Sometimes used as a name for temporary or insignificant variables (“don’t care”). Also, it represents the result of the last expression in a Python REPL session.

In [70]:
for _ in range(2):
    print("Hello")
print(_)

Hello
Hello
1


In [71]:
car = ("red", "mango", 12, 3.5, 456)
color, _, number, _, fl = car

print(color)
print(_)

red
3.5


In Python REPL, the single underscore `_` is used to represent the result of the last expression evaluated. For example, 

```python
>>> 1 + 2
3
>>> _ + 3
6
```

It is also handy if you are constructing objects on the fly and interacting with assigning them a name
```python
>>> list()
[]
>>> _.append(1)
>>> _.append(2)
>>> _
[1, 2]


### Summary

- **Single Leading Underscore `_var`**: Naming convention indicating a name is meant for internal use. A hint for programmers and not enforced by the interpreter (except in wildcard imports).

- **Single Trailing Underscore `var_`**: Used by convention to avoid naming conflicts with Python keywords.

- **Double Leading Underscore `__var`**: Triggers name mangling when used in a class context. Enforced by the Python interpreter.

- **Double Leading and Trailing Underscore `__var__`**: Indicates special methods defined by the Python language. Avoid this naming scheme for your own attributes.

- **Single Underscore `_`**: Sometimes used as a name for temporary or insignificant variables (“don’t care”). Also, it represents the result of the last expression in a Python REPL session.

## 2.5 String Formatting

Now, we will go through four different ways to format strings in Python. Each method has its own advantages and disadvantages, and the best method to use will depend on your specific use case.

1. Old-style string formatting using `%` operator
2. New-style string formatting using `str.format()`
3. f-strings (formatted string literals)
4. Template strings using `string.Template` class

### 2.5.1 Old-style string formatting using `%` operator
The old-style string formatting using `%` operator is the oldest method of string formatting in Python. It is similar to the `printf` function in C and uses format specifiers to indicate where to insert values into the string.


In [ ]:
errorno = 50159747054
name = "Bob"

In [ ]:
"Hello, %s" % name

'Hello, Bob'

In [ ]:
"Hey %s, You have received %x error in your code" % (name, errorno)

'Hey Bob, You have received badc0ffee error in your code'

In [ ]:
"Hey %(name)s, there is an error with code %(errorcode)x" % {
    "name": name,
    "errorcode": errorno,
}

'Hey Bob, there is an error with code badc0ffee'

### 2.5.2 New-style string formatting using `str.format()`

The new-style string formatting using `str.format()` was introduced in Python 3 and later backported to Python 2.7. It is more powerful and flexible than the old-style string formatting and allows you to use named placeholders, positional arguments, and format specifiers.

In [ ]:
"hello {}, your error code is {}".format(name, errorno)

'hello Bob, your error code is 50159747054'

In [ ]:
"hello {name},your error code is 0x{code:x}".format(code=errorno, name=name)

'hello Bob,your error code is 0xbadc0ffee'

### 2.5.3 f-strings (formatted string literals)

f-strings, or formatted string literals, were introduced in Python 3.6. They provide a more concise and readable way to include expressions inside string literals. To create an f-string, simply prefix the string with the letter `f` or `F`, and use curly braces `{}` to include expressions.


In [ ]:
f"Hello {name}"

'Hello Bob'

In [ ]:
# you can also embed python expressions

f"when you add 5 to 2, you get {5+2}"

'when you add 5 to 2, you get 7'

Behind the scenes, formatted string literals are a python parser feature that converts f-strings into a series of string constants and expressions. Then they get joined up to build the final string. 

In [ ]:
import dis


def greet(name, question):
    return f"Hello {name}, your question is {question}"


dis.dis(greet)

  3           0 RESUME                   0

  4           2 LOAD_CONST               1 ('Hello ')
              4 LOAD_FAST                0 (name)
              6 FORMAT_VALUE             0
              8 LOAD_CONST               2 (', your question is ')
             10 LOAD_FAST                1 (question)
             12 FORMAT_VALUE             0
             14 BUILD_STRING             4
             16 RETURN_VALUE


In [ ]:
f"Hey {name}, there is an error with code {errorno:#x}"

'Hey Bob, there is an error with code 0xbadc0ffee'

### 2.5.4 Template strings using `string.Template` class


This is less powerful but useful in some cases. The `string.Template` class provides a way to create simple string templates with placeholders that can be replaced with values.

In [ ]:
from string import Template

t = Template("Hey, $name!")

t.substitute(name=name)

'Hey, Bob!'

The best use case for `string.Template` is when you are handling format strings that are user-generated or come from an untrusted source. 

In [ ]:
# let us look at example of possible exploit that can be avoided using string.Template


SECRET = "this-is-a-secret"


class Error:
    def __init__(self):
        pass


errno = Error()

user_input = "{error.__init__.__globals__[SECRET]}"


user_input.format(error=errno)

'this-is-a-secret'

In [ ]:
# correct way of doing it is

user_input = "${error.__init__.__globals__[SECRET]}"

Template(user_input).substitute(error=errno)

ValueError: Invalid placeholder in string: line 1, col 1

> Which string formatting to use?

If your format strings are user supplied, use Template strings to avoid security issues. Otherwise, use Literal String Interpolation (f-strings) if you are using Python 3.6 or later. If you are using Python 2.7 or earlier, use `str.format()` method. Avoid using old-style string formatting with `%` operator as it is less readable and less powerful than the other methods.

## 2.6 Zen of Python

The Zen of Python is a collection of guiding principles for writing computer programs in the Python language. It was written by Tim Peters and is included as an Easter egg in the Python interpreter. You can access it by running the following command in a Python shell:

```python
import this
```

In [ ]:
import this

The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!
